In [1]:
%matplotlib inline

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

In [ ]:
users = pd.read_csv("users.dat", sep="::", header=None)

movies = pd.read_csv("movies.dat", sep="::", header=None)

ratings = pd.read_csv("ratings.dat", sep="::", header=None)

print(users.info())
print(movies.info())
print(ratings.info())

c:\users\a\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
c:\users\a\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until
c:\users\a\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (se

In [ ]:
usersHeader = ["userID", "gender", "age", "occupation", "zipCode"]

# age to float num (mean)  and to factor

moviesHeader_init = ["movieID", "title", "genres"]
moviesHeader = ["movieID", "year", "title", "genres"]
# genres must be ONE HOT encoding



ratingsHeader = ["userID", "movieID", "rating", "time"]

In [ ]:
movies.columns=moviesHeader_init
movies.head()
procMovies=pd.DataFrame(data={'movieID':movies['movieID']})
x =movies['title'].str.extract('(.*)\((\d{4})\)$')
procMovies['title']=x[0]
procMovies['year']=x[1]
procMovies['genres']=movies['genres'].str.split('|')
procMovies['howManyGenres']=procMovies['genres'].str.len()
print(procMovies.head())

procMovies1Hot=procMovies.copy()
procMovies1Hot=procMovies1Hot.drop('genres', 1).join(pd.get_dummies(pd.DataFrame(procMovies1Hot['genres'].tolist()).stack()).astype(int).sum(level=0))
print(procMovies1Hot.head())


procMoviesUniformHot=procMovies1Hot.copy()
procMoviesUniformHot.iloc[:,4:]=procMoviesUniformHot.iloc[:,4:].div(procMoviesUniformHot['howManyGenres'],axis=0)
print(procMoviesUniformHot.head())

In [ ]:
# how many genres does a movie have?
g=procMovies.groupby('howManyGenres').size()
print(g)
procMovies.hist(column='howManyGenres')
# movies with the greates number of genres
procMovies.sort_values('howManyGenres',ascending=False).head()

In [ ]:
# oldest movies
print(procMovies.sort_values('year').head())
# newest movies
print(procMovies.sort_values('year',ascending=False).head())
# how many movies for each year do we have?
g=procMovies.groupby('year').size()
g.plot.bar(figsize=(15,10))


In [ ]:
# in how many movies does each genre appear?
g=procMovies1Hot.iloc[:,4:].sum().sort_values()
g.plot.barh()

# in how many movies does each genre appear (goup by year)?
g=procMovies1Hot.columns[4:].tolist()
g.append('year')
g=procMovies1Hot[g].groupby('year').sum()#.sort_values()
g.plot.bar(figsize=(20,7))

In [ ]:
# how much is used a genre?
# the usage og a genre is a value in [0,1]
# if a movie has both thriller and action then thriller is used for 0.5 and action for 0.5
# if a movie has only thriller then thriller is used for 1
g=procMoviesUniformHot.iloc[:,4:].sum().sort_values()
g.plot.barh()
## for drama and comedy the plot is almost the same as before and they are at the top
## but this does not happen for action, thriller, romance
## the difference between the two plots is that for the second one we are giving less weight to a genre if there are also other genres
## so it seems that movies with drama or comedy as genre has only 1 genre
## instead movies with action, thriller, romance as genre has more than 1 genre

In [ ]:
# let's verify the reasoning written above:
## comedy and drama are usually the unique genre of the movie
## instead action,thriller, romance are mixed with other genres
genres=procMovies1Hot.columns[4:].tolist() # get list of genres
# group by howManyGenres and count how many movies for each genre
g=procMovies1Hot.groupby('howManyGenres')[genres].sum()
t=g.transpose() # a genre for each row
# each genre is used in movies with 1,2,.. genres,
## column 1 has how many movies used the given genre has unique genre,
## column 2 has how many movies used the given genre together with another genre
## column N has how many movies usd the given genre together with N-1 genres

# make a dataframe with columns: genre, howManyGenres, numberOfMovies
t['genre']=t.index
q=t.melt(id_vars='genre',value_name='numberOfMovies')
q['howManyGenres']=q['howManyGenres'].astype(int) # howManyGenres is of type int
# for each genre compute weighted mean of howManyGenre with numberOfMovies as weight 
w=q.groupby('genre').agg(lambda x: np.average(x['howManyGenres'],weights=x['numberOfMovies'])).iloc[:,1]
t['wmHowManyGenre']=w
t=t.sort_values('wmHowManyGenre')
t.iloc[:,0:6].plot.barh(figsize=(10,7),width=1,cmap='RdBu_r')
t
## so drama is the second genre used alone in a movie (after documentary)
## then there is horror and comedy
## instead in average action, thriller and romance are used together with at least 2 genres
## fantasy and adventure are the genres that are more mixed with the other ones

In [ ]:
## let's check for each genre wich is the genre used together with it
## make dataframe with 2 columns: movieId and genre
## for movies with N genres there are N rows
q=procMovies.set_index('movieID').genres.apply(pd.Series).stack().reset_index(level=-1, drop=True).reset_index()
q.columns=('movieID','genre')
q=q.set_index('movieID')
procMovies1Hot=procMovies1Hot.set_index('movieID')
print(q.shape)

# for each genre (that is a row) you have the number of movies that used it together with Action (Action column), Adventure (Adventure column) and so on
g=procMovies1Hot.join(q)
g.head()
g.shape
q=g.groupby('genre')[genres].sum()
q


In [ ]:
## divide the values by the number of movies for that genre, in order to have relative values and so compare different genres
import seaborn as sns
qr=q.div(q.sum(axis=1), axis=0)
## the lighter the square more movies with the two genres together
sns.heatmap(qr)
plt.show()
## the white squares means that the two genres are almost never mixed together
qrl=qr.apply(np.log)
sns.heatmap(qrl)
plt.show()
## in the first plot
##  Documentary-Documentary is white, in fact Documentary is almost never mixed with other genres
##  Musical-(Aniation,Children's,Comedy) is quite light so Musical is usually mixed with them

## in the second plot
##  Action-(Documentary,Film Noir) is white, in fact Action is almost never mixed with them
##  Children-(Crime,Documentary,Film Noir) is white, in fact Children's is almost never mixed with them
##    (strange that there is not War, Children and War are used together in the movie with 6 genres, Transformners 1986 and another one)
##  Documentary-(Comedy,Drama,Musical,War) is black, Documentary is mixed with only these genres
##  Comedy and Drama are mixed with all the different genres